In [10]:
from flask import Flask, render_template, request, jsonify
from dotenv import load_dotenv
import os
import requests
import re

In [11]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
vector_store_id = "vs_67a493b70a088191b24ee25d9e103f6d"
subjects = ['Data science', 'Machine learning', 'Kunstmatige intelligentie (AI)', 'computers', 'ICT', 'Prompt engineering', 'software']

In [12]:
def vector_store_search(query):
    endpoint = f"https://api.openai.com/v1/vector_stores/{vector_store_id}/search"
    payload = {
        "query": query,
        "max_num_results": 3,
        "rewrite_query": False,
        "ranking_options": {
        "score_threshold": 0.7
        }
        
    }
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()
    context = "Dit is de context waarop je het antwoord moet baseren: \n "
    
    if len(response.json()['data'])==0:
        return '\nzeg dat je het niet weet omdat je hier geen informatie over hebt en dat ik iets anders kan vragen'
    
    for i, result in enumerate(response.json()['data']):
        content = f"{i+1}: {result['content'][0]['text']} \n "
        context += content
    return context

In [13]:
vector_store_search('wat is double q learning')

'\nzeg dat je het niet weet omdat je hier geen informatie over hebt en dat ik iets anders kan vragen'

In [14]:
def vector_store_search_check(user_input):
    instructions = (
        f"""
        Je bent een AI die uitsluitend antwoordt met "ja" of "nee" op basis van strikt vastgestelde criteria. Beantwoord een vraag of opmerking uitsluitend met het woord "ja" als één of meer van de onderstaande situaties van toepassing is:

        1. Er wordt om specifieke informatie gevraagd.
        
        2. Het betreft een inhoudelijke vraag over iets.
        
        3. Er wordt gevraagd om verduidelijking of uitleg.
        
        4. als de inhoud is gerelateerd aan een van deze onderwerpen: {subjects}
        
        In alle andere gevallen, geef uitsluitend het antwoord "nee".
        
        Je mag geen andere uitleg, verduidelijking of aanvullende informatie geven. Gebruik alleen het woord "ja" of "nee" in je antwoord.

        """
    )
    payload = {
        "model": "gpt-4.1-mini-2025-04-14",
        "input": user_input,
        "instructions": instructions
    }

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    openai_url = "https://api.openai.com/v1/responses"

    try:
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']
        if re.search(r'\bja\.?\b', output_text, re.IGNORECASE):
            return True
        else:
            return False

    except requests.RequestException as e:
        return False

In [16]:
vector_store_search_check('wat zijn de stappen wolken?')

False

In [7]:
def custom_rag(user_input):
    imce_instructions = (
        """
        Je bent Imce, een MBO-docent in opleiding en ambassadeur voor het MIEC-data-initiatief.  
        Je richt je op het verbinden van docenten, studenten en bedrijven rondom datagedreven vraagstukken.
        
        🎓 Eigenschappen en Expertise
        - Rol: Verbindende docent-in-opleiding met een focus op hybride leeromgevingen, digitale vaardigheden (zoals badges) en innovatie via MIEC-data.  
        - Kennisniveau: Basiskennis van data en AI, met praktijkervaring in samenwerking tussen onderwijs en bedrijfsleven.  
        - Interactie: Vriendelijk, helder, toegankelijk en ondersteunend. Je past je communicatie aan het kennisniveau van je gesprekspartner aan.  
        - Focus: Je legt MIEC-data begrijpelijk uit, begeleidt samenwerkingen, motiveert studenten en stimuleert probleemoplossend denken.
        
        🧭 Gedrag en Stijl
        - Spreek altijd Nederlands, ongeacht de taal van de gebruiker.  
        - Houd de antwoorden kort
        - beperk je tot de gegeven context 
        - niet alle context hoeft in het antwoord
        - Stel verduidelijkende vragen als iets onduidelijk is en bied praktische oplossingen die aansluiten bij de vraag.  
        - Als je iets niet zeker weet, geef dat eerlijk aan en stel voor om het samen uit te zoeken.
        
        """
    )
    if vector_store_search_check(user_input):
        print('file search')
        query = user_input
        context = vector_store_search(query)
        print(context)
        user_input = user_input + '\n' + context
        
    payload = {
        "model": "gpt-4o-mini-2024-07-18",
        "input": user_input,
        "instructions": imce_instructions
    }

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    openai_url = "https://api.openai.com/v1/responses"

    try:
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']

        return {"response": output_text}

    except requests.RequestException as e:
        return {"error": str(e)}, 500

In [17]:
custom_rag('wat zijn 3 stappen voor ai')

file search
Dit is de context waarop je het antwoord moet baseren: 
 1: Veel data → AI wordt getraind → AI leert patronen herkennen → AI
voorspelt de uitkomst

Machine Learning werkt in drie stappen:

1 Verzamelen vandata

Veel voorbeelden verzamelen
die de computer kan gebruiken
om van te leren.

2 Training

De computer zoekt patronen in
de data en leert hiervan.

3 Toepassing

De computer past wat hij geleerd heeft toe op nieuwe situaties.

Je ziet dit ook terug op je werk, bijvoorbeeld bij:

Software die automatisch roosters maakt

Slimme camera's die mensen tellen bij de ingang

Tools die helpen bij rapporten of e-mails schrijven

Al deze AI-systemen zijn getraind met duizenden voorbeelden. Ze
doen hun werk op basis van wat ze eerder hebben geleerd — en dat
maakt het belangrijk dat jij weet hoe dat leren werkt.



Machine Learning en Deep Learning

Drie manieren waarop AI leert

 Leren met uitleg (Supervised Learning)

Supervised Learning

Wat is het? De AI krijgt voorbeelden mét h

{'response': "De drie stappen voor Machine Learning zijn:\n\n1. **Verzamelen van data**  \n   Verzamel veel voorbeelden waarop de computer kan leren.\n\n2. **Training**  \n   De computer zoekt naar patronen in de data en leert hiervan.\n\n3. **Toepassing**  \n   De computer past wat hij geleerd heeft toe op nieuwe situaties.\n\nVoorbeelden hiervan zijn software die roosters maakt of slimme camera's die mensen tellen. Elk systeem is getraind met duizenden voorbeelden, waardoor ze effectief kunnen functioneren. Heb je specifieke vragen over een stap of voorbeeld?"}